In [4]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import poisson
from seirsplus.models import *
from network_utils import *
from stats_utils import *
from intervention_utils import *
from camp_params import *
import pickle as pkl

In [5]:
graph, nodes_per_struct = load_graph("Moria_wNeighbors")

In [6]:
food_weight = 0.407
graph_1fq = connect_food_queue(graph, nodes_per_struct, food_weight, "food")

In [7]:
node_groups=create_node_groups(graph)

In [8]:
transmission_rate = 1.28
progression_rate = round(1/5.1, 3)
recovery_rate = 0.056 # Approx 1/18 -> Recovery occurs after 18 days
hosp_rate = round(1/11.4, 3) #1/6.3 # From Tucker Model
# crit_rate = 0.3 # From camp_params
crit_rate = list((sample_pop["death_rate"] / sample_pop["prob_symptomatic"]) / sample_pop["prob_hospitalisation"])
death_rate = 0.75

prob_global_contact = 1
prob_detected_global_contact = 1

# prob_hosp_to_critical = list(sample_pop["death_rate"]/sample_pop["prob_hospitalisation"])
prob_death = list(sample_pop["death_rate"])
prob_asymptomatic = list(1 - sample_pop["prob_symptomatic"])
prob_symp_to_hosp = list(sample_pop["prob_hospitalisation"])

init_symp_cases = 1
init_asymp_cases = 0

In [9]:
# Model construction
ref_model = SymptomaticSEIRSNetworkModel(G=graph_1fq, beta=transmission_rate, sigma=progression_rate, gamma=recovery_rate, 
                                         lamda=progression_rate, mu_H=crit_rate, eta=hosp_rate, p=prob_global_contact, a=prob_asymptomatic, f=death_rate, 
                                         h=prob_symp_to_hosp, q=prob_detected_global_contact, initI_S=init_symp_cases, initI_A=init_asymp_cases, store_Xseries=True,
                                        node_groups=node_groups)

In [10]:
def run_simulation_group(model, t, print_info=False, store_every=1):
    simulation_results = defaultdict(list)

    print(f"Running simulation for {t} steps...\n")

    for i in tqdm(range(t + 1)):
        model.run(T=1, verbose=print_info)

        if i % store_every == 0:
            # Store the quantities of the last time step t 
            simulation_results["Symptomatic"].append(model.numS[-1])
            simulation_results["Exposed"].append(model.numE[-1])
            simulation_results["Infected_Presymptomatic"].append(model.numI_pre[-1])
            simulation_results["Infected_Symptomatic"].append(model.numI_S[-1])
            simulation_results["Infected_Asymptomatic"].append(model.numI_A[-1])
            simulation_results["Hospitalized"].append(model.numH[-1])
            simulation_results["Recovered"].append(model.numR[-1])
            simulation_results["Fatalities"].append(model.numF[-1])
            simulation_results["Detected_Presymptomatic"].append(model.numD_pre[-1])
            simulation_results["Detected_Symptomatic"].append(model.numD_S[-1])
            simulation_results["Detected_Asymptomatic"].append(model.numD_A[-1])
            simulation_results["T_index"].append(model.tidx)
    
    group_data=model.nodeGroupData
    for group,group_detail in group_data.items():
        for key,value in group_detail.items():
            if key=='numS':
                simulation_results[f'Symptomatic {group}']=list(value[simulation_results["T_index"]])
            elif key=='numE':
                simulation_results[f'Exposed {group}']=list(value[simulation_results["T_index"]])
            elif key=='numI_pre':
                simulation_results[f'Infected_Presymptomatic {group}']=list(value[simulation_results["T_index"]])
            elif key=='numI_S':
                simulation_results[f'Infected_Symptomatic {group}']=list(value[simulation_results["T_index"]])
            elif key=='numI_A':
                simulation_results[f'Infected_Asymptomatic {group}']=list(value[simulation_results["T_index"]])
            elif key=='numH':
                simulation_results[f'Hospitalized {group}']=list(value[simulation_results["T_index"]])
            elif key=='numR':
                simulation_results[f'Recovered {group}']=list(value[simulation_results["T_index"]])
            elif key=='numF':
                simulation_results[f'Fatalities {group}']=list(value[simulation_results["T_index"]])
            elif key=='numD_E':
                simulation_results[f'Detected_Exposed {group}']=list(value[simulation_results["T_index"]])
            elif key=='numD_pre':
                simulation_results[f'Detected_Presymptomatic {group}']=list(value[simulation_results["T_index"]])
            elif key=='numD_S':
                simulation_results[f'Detected_Symptomatic {group}']=list(value[simulation_results["T_index"]])
            elif key=='numD_A':
                simulation_results[f'Detected_Asymptomatic {group}']=list(value[simulation_results["T_index"]])
    return simulation_results

In [11]:
t_steps = 20
simulation_results = run_simulation_group(ref_model, t_steps)

  5%|▍         | 1/21 [00:00<00:02,  6.79it/s]

Running simulation for 20 steps...



100%|██████████| 21/21 [05:19<00:00, 15.23s/it]


In [14]:
output_df = results_to_df(simulation_results)

In [15]:
output_df

,Symptomatic,Exposed,Infected_Presymptomatic,Infected_Symptomatic,Infected_Asymptomatic,Hospitalized,Recovered,Fatalities,Detected_Presymptomatic,Detected_Symptomatic,...,Infected_Symptomatic age>80,Infected_Asymptomatic age>80,Hospitalized age>80,Recovered age>80,Fatalities age>80,Detected_Exposed age>80,Detected_Presymptomatic age>80,Detected_Symptomatic age>80,Detected_Asymptomatic age>80,Time
0,18695.0,3.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,18688.0,8.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
2,18683.0,11.0,4.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
3,18674.0,18.0,5.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
4,18663.0,26.0,7.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
5,18649.0,34.0,12.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6
6,18634.0,42.0,15.0,5.0,3.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7
7,18589.0,71.0,29.0,4.0,5.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8
8,18550.0,97.0,40.0,5.0,6.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9
9,18490.0,138.0,51.0,6.0,13.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10


In [9]:
n_pop = 18700

# Grid info for isoboxes
dims_isoboxes = (29, 28)  # 812

# Grid info for tents
dims_block1 = (20, 67)  # 1340
dims_block2 = (53, 15)  # 795
dims_block3 = (19, 28)  # 532

# Isoboxes
pop_isoboxes = 8100
pop_per_isobox = 10
n_isoboxes = dims_isoboxes[0]*dims_isoboxes[1]

# Tents
n_tents = 2650
pop_tents = 10600
pop_per_tent = 4

# Others 
n_bathrooms = 144
n_ethnic_groups = 8

# We define neighboring structures within a range of 2 in the structure grid
proximity = 2

# Define the maximum population per structures (tents and isoboxes) drawn from a poisson distribution
max_pop_per_struct = list(poisson.rvs(mu=pop_per_isobox, size=n_isoboxes))
max_pop_per_struct = max_pop_per_struct + list(poisson.rvs(mu=pop_per_tent, size=dims_block1[0]*dims_block1[1]))
max_pop_per_struct = max_pop_per_struct + list(poisson.rvs(mu=pop_per_tent, size=dims_block2[0]*dims_block2[1]))
max_pop_per_struct = max_pop_per_struct + list(poisson.rvs(mu=pop_per_tent, size=dims_block3[0]*dims_block3[1]))

n_structs = len(max_pop_per_struct)

# Sample the population age, and parameter rates
sample_pop = sample_population(n_pop, "data/augmented_population.csv")

transmission_rate = 1.28
progression_rate = 1/5.1
recovery_rate = 0.056 # Approx 1/18 -> Recovery occurs after 18 days
hosp_rate = 1/11.4 #1/6.3 # From Tucker Model
crit_rate = 0.3 # From camp_params

prob_global_contact = 1
prob_detected_global_contact = 1



prob_hosp_to_critical = list(sample_pop["death_rate"]/sample_pop["prob_hospitalisation"])
prob_asymptomatic = list(1 - sample_pop["prob_symptomatic"])
prob_symp_to_hosp = list(sample_pop["prob_hospitalisation"])

init_symp_cases = 1
init_asymp_cases = 0

In [52]:
ref_model = SymptomaticSEIRSNetworkModel(G=graph, beta=transmission_rate, sigma=progression_rate, gamma=recovery_rate, 
                                         lamda=progression_rate, mu_H=crit_rate, eta=hosp_rate, p=prob_global_contact, a=prob_asymptomatic, f=prob_hosp_to_critical, 
                                         h=prob_symp_to_hosp, q=prob_detected_global_contact, initI_S=init_symp_cases, initI_A=init_asymp_cases, store_Xseries=True,node_groups=node_groups)

In [53]:
node_states,simulation_results,grouped_data = run_simulation(ref_model, 20, print_info=False, store_every=1)

  0%|          | 0/20 [00:00<?, ?it/s]

Running simulation for 20 steps...



100%|██████████| 20/20 [02:50<00:00,  8.50s/it]


In [56]:
def find_whole_time(tseries, T):
    idt=[]
    for t in range(0, T + 1):
        idt.append(np.abs(tseries - t).argmin())
    return idt

In [51]:
def run_simulation(model, t, print_info=False, store_every=1):
    node_states = dict()
    simulation_results = defaultdict(list)

    print(f"Running simulation for {t} steps...\n")

    for i in tqdm(range(1, t + 1)):
        model.run(T=1, verbose=print_info)

        if i % store_every == 0:
            # Store the node states - an array of size (1, num_nodes) -> we store a copy because the array gets updated
            node_states[i] = np.copy(model.X.T)

            # Store the quantities of the last time step t 
            simulation_results["Symptomatic"].append(model.numS[-1])
            simulation_results["Exposed"].append(model.numE[-1])
            simulation_results["Infected_Presymptomatic"].append(model.numI_pre[-1])
            simulation_results["Infected_Symptomatic"].append(model.numI_S[-1])
            simulation_results["Infected_Asymptomatic"].append(model.numI_A[-1])
            simulation_results["Hospitalized"].append(model.numH[-1])
            simulation_results["Recovered"].append(model.numR[-1])
            simulation_results["Fatalities"].append(model.numF[-1])
            simulation_results["Detected_Presymptomatic"].append(model.numD_pre[-1])
            simulation_results["Detected_Symptomatic"].append(model.numD_S[-1])
            simulation_results["Detected_Asymptomatic"].append(model.numD_A[-1])
            simulation_results["T_index"].append(model.tidx)
    
    group_data=model.nodeGroupData
    time_stamps=model.tseries
    idt=find_whole_time(time_stamps,t)
    timed_group_data=group_data.copy()
    for group,group_detail in group_data.items():
        for key,value in group_detail.items():
            if key!='nodes' and  key!='mask' and key!='N':
                timed_group_data[group][key]=value[idt]
    
    
    return node_states,simulation_results,timed_group_data

In [1]:
#test for multiprocessing

In [1]:
from seirsplus.models import SymptomaticSEIRSNetworkModel
from network_utils import load_graph,create_node_groups,run_simulation_group_parallel,results_to_df_group,connect_food_queue
from params import ModelParams
import pickle as pkl
import pandas as pd
import dask
import os

In [2]:
num_iterations=10

In [3]:
lazy_sols=[]
graph, nodes_per_struct = load_graph("Moria_wNeighbors")
food_weight = 0.407
graph_1fq = connect_food_queue(graph, nodes_per_struct, food_weight, "food")
node_groups=create_node_groups(graph_1fq)
Params=ModelParams('model_params.csv','population_moria.csv')
# Model construction
for i in range(num_iterations):
    model = dask.delayed(SymptomaticSEIRSNetworkModel)(G=graph_1fq, beta=Params.transmission_rate, sigma=Params.progression_rate, gamma=Params.recovery_rate, 
                                           lamda=Params.progression_rate, mu_H=Params.crit_rate, eta=Params.hosp_rate, p=Params.prob_global_contact,a=Params.prob_asymptomatic, f=0.75, 
                                           h=Params.prob_symp_to_hosp, q=Params.prob_detected_global_contact, initI_S=Params.init_symp_cases, initI_A=Params.init_asymp_cases, store_Xseries=True,node_groups=node_groups)
    lazy_result = dask.delayed(run_simulation_group_parallel)(model,20)
    lazy_sols.append(lazy_result)

In [ ]:
%%time
sols = dask.compute(*lazy_sols)

In [14]:
graph, nodes_per_struct = load_graph("Moria_wNeighbors")
food_weight = 0.407
graph_1fq = connect_food_queue(graph, nodes_per_struct, food_weight, "food")
node_groups=create_node_groups(graph_1fq)
Params=ModelParams('model_params.csv','population_moria.csv')

In [15]:
%%time
# Model construction
sols=[]
for i in range(num_iterations):
    model = SymptomaticSEIRSNetworkModel(G=graph_1fq, beta=Params.transmission_rate, sigma=Params.progression_rate, gamma=Params.recovery_rate, 
                                           lamda=Params.progression_rate, mu_H=Params.crit_rate, eta=Params.hosp_rate, p=Params.prob_global_contact,a=Params.prob_asymptomatic, f=0.75, 
                                           h=Params.prob_symp_to_hosp, q=Params.prob_detected_global_contact, initI_S=Params.init_symp_cases, initI_A=Params.init_asymp_cases, store_Xseries=True,node_groups=node_groups)
    lresult = run_simulation_group(model,5)
    sols.append(lresult)

distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took

CPU times: user 54.5 s, sys: 29.5 s, total: 1min 24s
Wall time: 1min 21s


In [16]:
model.tseries

array([0.        , 0.25099808, 0.72580118, 0.75538034, 0.935942  ,
       1.17273042, 1.44570195, 1.53319341, 1.7810278 , 2.1163568 ,
       2.43022836, 3.10960159, 3.43532481, 3.5275245 , 3.63896462,
       3.77863688, 3.80059878, 3.85935965, 3.90143136, 3.91526014,
       3.91717043, 4.00287594, 4.01606412, 4.17324092, 4.30448459,
       4.31459192, 4.38454069, 4.51536405, 4.61733263, 4.67787288,
       4.70885955, 4.74099182, 4.8130183 , 4.83414221, 4.83818615,
       4.85401024, 4.91305313, 4.94844939, 4.95034183, 5.07530461,
       5.16668449, 5.28694074, 5.30384435, 5.37823335, 5.43213611,
       5.43404142, 5.45955256, 5.45986504, 5.55605057, 5.55700705,
       5.58546272, 5.62299624, 5.63338463, 5.7157354 , 5.71900493,
       5.72000959, 5.75844745, 5.79325858, 5.83432817, 5.85623403,
       5.99921542, 6.08051364])

In [ ]:
for i in range(num_iterations):
    simulation_results=sols[i]
    simulation_results['Run']=[i]*len(simulation_results)
    output_df = results_to_df_group(simulation_results,store=True,store_name=f"baseline_{i}.csv")

In [ ]:
sols[0]

In [ ]:
for i in range(num_iterations):
    simulation_results=sols[i]
    simulation_results['Run']=[i]*len(simulation_results)
    output_df = results_to_df_group(simulation_results,store=True,store_name=f"baseline_{i}.csv")